In [34]:
# jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000
import networkx as nx
import pandas as pd
import urllib
import numpy as np
import matplotlib.pyplot as plt 
import sys
import os
from networkx.algorithms import bipartite as bi



url = "https://raw.githubusercontent.com/Misterresearch/CUNY-Projects/master/tveanon_ip_guid.csv"
tvsubs = pd.read_csv(url, parse_dates=True, names=['IP','ID'], header=0)
print tvsubs

                  IP                                      ID
0      1.242.166.174  {21E4cC33-C623-F514-CcC1-44D7E28811E3}
1        1.42.46.226        96659c2c0045cdc1c729d8a4269a268c
2      100.0.115.126        ca1dc06138cda889adcfd017a55acc06
3        100.0.122.4        06efc69f1e71087e96d01a6f22cf7f5a
4      100.0.184.148  {744FAC75-30E3-5033-1cc0-5C996cCCCcc4}
5      100.0.197.165        f1969f8e6cff0c77e848ef2a3cccecaf
6       100.0.197.89  {ccEF74A6-4C3F-0Ac5-0Ec5-1cE5419125EE}
7      100.0.198.144        34e60674e957ff1161757790d7052630
8      100.0.205.254        86759e59c2eadc199a39f3c42ee20542
9      100.0.215.107        2f7f26d1f1aacca737cc88ecfae37014
10     100.0.221.240        68c0c55e9cf8c7750d98c71c6da43207
11       100.0.48.54        cd135d6ad29f3438cc9e4dd2ed9754e1
12        100.0.6.24        59c45a9cc14cd22960439d030c1d8f1d
13      100.0.62.144        ade331c5c6d07ec47f9f39eeeae8e2cc
14     100.1.102.192        30c727f149c92c8545817c8019f0ed9a
15      100.1.107.47    

In [35]:
B = nx.Graph()
B.add_nodes_from(tvsubs['IP'], bipartite=0)
B.add_nodes_from(tvsubs['ID'], bipartite=1)
B.add_edges_from(zip(tvsubs['IP'], tvsubs['ID']),weight=1)

#B is an unconnected graph
bottom_nodes, top_nodes = bi.sets(B)
top_nodes = set(n for n,d in B.nodes(data=True) if d['bipartite']==0)
bottom_nodes = set(B) - top_nodes

Bip = bi.weighted_projected_graph(B, top_nodes, ratio=False)
Bid = bi.weighted_projected_graph(B, bottom_nodes, ratio=False)
#nx.draw(Bip, node_size=0, node_color='w', edge_color='b', alpha=.2)
#plt.show()


## Setting Weights - Edges & Nodes

It'important to note that because we're developing subgraphs using the island_method from a projected graph, based on edge weight, using the "weight" setting in the add_edges_from function, will not work. It's actually the ratio argument (set to false), that gives us a weight setting for the edges in our projected graph that will actually give us the required parameters for our edge triming, base on the custom function found in Tsvetovat & Kouznetsov.

In [36]:
Bip.nodes()[:10]

['68.14.152.227',
 '216.220.65.94',
 '70.190.58.88',
 '68.9.124.26',
 '174.14.245.102',
 '172.58.105.169',
 '71.56.221.62',
 '99.67.94.41',
 '66.91.58.49',
 '208.58.214.228']

In [37]:
Bip.edges(data=True)[:10]

[('74.112.84.54', '74.112.84.169', {'weight': 1}),
 ('69.140.180.252', '208.54.45.170', {'weight': 1}),
 ('174.248.150.8', '166.147.246.124', {'weight': 1}),
 ('174.248.150.8', '107.77.229.174', {'weight': 1}),
 ('70.44.161.2', '68.48.117.88', {'weight': 1}),
 ('74.94.201.211', '74.94.202.7', {'weight': 1}),
 ('74.94.201.211', '74.94.201.161', {'weight': 1}),
 ('74.90.204.255', '74.90.206.244', {'weight': 1}),
 ('74.90.204.255', '74.90.205.57', {'weight': 1}),
 ('74.85.204.112', '74.85.202.228', {'weight': 1})]

In [38]:
degree = nx.degree(Bip)
d =[]
for key, value in sorted(degree.iteritems(), key=lambda (k,v): (v,k), reverse = True):
    d.append((key, value))
    
df=pd.DataFrame(d, columns=('IP','Degree'))
df[:10]

,IP,Degree
0,44.205.194.218,254
1,64.144.56.10,36
2,96.8.194.94,34
3,96.8.194.85,34
4,96.8.194.84,34
5,96.8.194.80,34
6,96.8.194.8,34
7,96.8.194.79,34
8,96.8.194.76,34
9,96.8.194.74,34


## Water Levels - Node Centrality vs Edges

Although we're going to use edge weight to develop our subgraphs, the table above provides a ranking of our projected graph 'IP' nodes based on degree centrality counts, which could also be a viable option. Even with this ranker, there's a strong business application here. The next two tables are an adjancy table and nodes for our projected graph with associated weights. Again, for this specific analysis we've chosen to develop our subgraphs using edge weights instead. 

In [39]:
Bip.remove_nodes_from(nx.isolates(Bip)) 
#Bip.nodes()
m = nx.to_pandas_dataframe(Bip, weight=1)
m [:10]

,74.112.84.54,69.140.180.252,174.248.150.8,70.44.161.2,74.94.201.211,74.90.204.255,74.85.204.112,66.87.147.245,74.249.157.65,107.77.194.129,...,65.74.244.248,162.242.26.96,209.122.202.151,208.184.54.70,70.211.65.100,75.20.160.245,74.110.54.66,174.101.209.140,174.248.144.40,50.202.144.242
74.112.84.54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69.140.180.252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
174.248.150.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70.44.161.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74.94.201.211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74.90.204.255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74.85.204.112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66.87.147.245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74.249.157.65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107.77.194.129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
Bip.degree(weight='weight').items()[:10]

[('74.106.74.144', 5),
 ('184.188.42.10', 1),
 ('66.87.65.9', 3),
 ('68.70.186.217', 1),
 ('174.248.147.65', 1),
 ('74.94.201.211', 2),
 ('66.87.147.245', 4),
 ('75.188.215.80', 1),
 ('107.200.4.186', 1),
 ('147.0.48.122', 1)]

In [41]:
Bip2=Bip.copy()
def trim_edges(Bip, weight=1):
    Bip2=nx.Graph()
    for f,to,edata in Bip.edges(data=True):
                if edata['weight'] > weight:
                        Bip2.add_edge(f,to,edata)
    return Bip2
trim_edges(Bip2, 2)
#E.edges()
Bip2.edges(data=True)[0:10]

[('74.106.74.144', '172.58.6.247', {'weight': 1}),
 ('74.106.74.144', '74.106.74.70', {'weight': 1}),
 ('74.106.74.144', '74.106.90.148', {'weight': 1}),
 ('74.106.74.144', '172.58.6.149', {'weight': 1}),
 ('74.106.74.144', '162.207.227.144', {'weight': 1}),
 ('184.188.42.10', '64.144.226.68', {'weight': 1}),
 ('66.87.65.9', '66.87.64.245', {'weight': 1}),
 ('66.87.65.9', '66.87.64.147', {'weight': 1}),
 ('66.87.65.9', '66.87.64.99', {'weight': 1}),
 ('68.70.186.217', '72.240.145.98', {'weight': 1})]

In [47]:
Bip3=Bip.copy()
def trim_nodes(Bip, weight=1):
    #Bip3=nx.Graph()
    for i,j, in Bip.degree(weight='weight').items():
                if j > weight:
                        Bip3.add_node(i)
    return Bip3
trim_nodes(Bip3, 1)
Bip3.degree(weight='weight').items()[:10]

[('74.106.74.144', 5),
 ('184.188.42.10', 1),
 ('68.70.186.217', 1),
 ('66.87.150.64', 2),
 ('174.248.147.65', 1),
 ('74.94.201.211', 2),
 ('66.87.147.245', 4),
 ('107.200.4.186', 1),
 ('108.27.220.126', 1),
 ('147.0.48.122', 1)]

In [43]:
def island_method(Bip, iterations=3):
    weights= [edata['weight'] for f,to,edata in Bip.edges(data=True)]
    mn=int(min(weights))
    mx=int(max(weights))
    #compute the size of the step, so we get a reasonable step in iterations
    step=int((mx-mn)/iterations)
    return [[threshold, trim_edges(Bip, threshold)] for threshold in range(mn,mx,step)]
island_method(Bip, 3)


[[1, <networkx.classes.graph.Graph at 0x1115bf850>],
 [2, <networkx.classes.graph.Graph at 0x11301d490>],
 [3, <networkx.classes.graph.Graph at 0x11118da50>],
 [4, <networkx.classes.graph.Graph at 0x120ed9210>]]

In [44]:
#list(nx.connected_components(Bip))
#sorted(nx.connected_component_subgraphs(Bip), key = len, reverse=True)

In [45]:
islands = island_method(Bip)
islands
len(islands)
for i in islands:
    print i[0], len(i[1]), len(list(nx.connected_components(i[1])))

1 119 35
2 44 14
3 19 7
4 7 3


## Code Sources:

[https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.algorithms.bipartite.projection.weighted_projected_graph.html](Weighted Parameter in Projected Graph)

[https://stackoverflow.com/questions/27878034/networkx-get-degree-of-node-with-weights-applied](Transforming Degree Centrality to Node Weights)

[https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.Graph.edges.html](Assigning Weights to Graph Edges)

[http://networkx.github.io/documentation/networkx-1.9/reference/api_1.9.html#miscellaneous-changes](Networkx: connected_component_subgraphs have been removed)

[https://github.com/rian39/data_forms/blob/master/ml_lit/ML_literature.py](An Island Method Subgraph Generation Technique)